# Classical results
---
In this notebook, we obtain states of spin networks by contractions of tensors, using package ```tensornetwork```. Results of this method are used to verify fidelity of states obtained using qunatum methods.

All names used in this notebook are the same as in the  article and all references are refrences to the article.

In [1]:
import tensornetwork as tn
import numpy as np

We define tensor representing node $\iota_k^{m_1m_2m_3m_4}$, which, in our case of spins $\frac{1}{2}$, has shape $2\times2\times2\times2\times2$.

In [2]:
iota = np.array([[[[[ 0.        ,  0.        ],
                    [ 0.        ,  0.        ]],
                   
                   [[ 0.        ,  0.5       ],
                    [-0.5       ,  0.        ]]],

                  [[[ 0.        , -0.5       ],
                    [ 0.5       ,  0.        ]],

                   [[ 0.        ,  0.        ],
                    [ 0.        ,  0.        ]]]],

                 [[[[ 0.        ,  0.        ],
                    [ 0.        ,  0.57735027]],

                   [[ 0.        , -0.28867513],
                    [-0.28867513,  0.        ]]],

                  [[[ 0.        , -0.28867513],
                    [-0.28867513,  0.        ]],

                   [[ 0.57735027,  0.        ],
                    [ 0.        ,  0.        ]]]]])

We define also tensor representing state of link $\alpha_{m_1m_2}$. In our case, singlet state $\frac{1}{\sqrt{2}}\left(|01\rangle-|10\rangle\right)$.

In [3]:
link = np.array([0,1/np.sqrt(2),-1/np.sqrt(2),0]).reshape((2,2))

In [4]:
def norm(state):
    '''Return normalized state'''
    state_conj = tn.replicate_nodes([state], conjugate=True)[0]
    for i in range(len(state.edges)):
        state.edges[i]^state_conj.edges[i]
    n = tn.contractors.auto([state, state_conj]).tensor
    state.tensor = state.tensor/np.sqrt(n)
    return state

# Open node

Construction of node with four free links

$$\left(\Gamma_2'\right)_{km_1'm_2'm_3'm_4'}=\iota_k^{m_1m_2m_3m_4}\alpha_{m_1m_1'}\alpha_{m_2m_2'}\alpha_{m_3m_3'}\alpha_{m_4m_4'}$$

In [5]:
node = tn.Node(iota)
links = [tn.Node(link) for i in range(4)]
node[1]^links[0][0]
node[2]^links[1][0]
node[3]^links[2][0]
node[4]^links[3][0]
state = tn.contractors.auto([node]+links, output_edge_order=[node[0]]+[l[1] for l in links])
state_open_node = norm(state)
state_open_node.tensor.reshape(2**5)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.35355339, -0.35355339,  0.        ,  0.        , -0.35355339,
        0.35355339,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.40824829,
        0.        , -0.20412414, -0.20412414,  0.        ,  0.        ,
       -0.20412414, -0.20412414,  0.        ,  0.40824829,  0.        ,
        0.        ,  0.        ])

# Pentagram from open node

Construction of pentagram using open node and six links:

$$
\left(\Gamma_5\right)_{kk_1k_2k_3k_4}=
\left(\Gamma_2'\right)_{km_1n_1s_1t_1}

\iota_{k_1}^{m_1m_2m_3m_4}
\iota_{k_2}^{n_1n_2n_3n_4}
\iota_{k_3}^{s_1s_2s_3s_4}
\iota_{k_4}^{t_1t_2t_3t_4}
              
\alpha_{m_2n_2}
\alpha_{s_4t_4}
\alpha_{m_4t_2}
\alpha_{n_3s_3}
\alpha_{m_3s_2}
\alpha_{n_4t_3}
$$

In [6]:
links = [tn.Node(link) for i in range(6)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_node[1]^n[0][1]
state_open_node[2]^n[1][1]
state_open_node[3]^n[2][1]
state_open_node[4]^n[3][1]

links[0][0]^n[0][2]
links[0][1]^n[1][2]

links[4][0]^n[0][3]
links[4][1]^n[2][2]

links[2][0]^n[0][4]
links[2][1]^n[3][2]

links[3][0]^n[1][3]
links[3][1]^n[2][3]

links[5][0]^n[1][4]
links[5][1]^n[3][3]

links[1][0]^n[2][4]
links[1][1]^n[3][4]

state = tn.contractors.auto([state_open_node]+n+links, output_edge_order=[state_open_node[0],n[0][0],n[1][0],n[2][0],n[3][0]])
state_pentagram = norm(state)
state_pentagram.tensor.reshape(2**5)

array([ 4.00891867e-01, -1.11486036e-35,  7.81587279e-37,  4.00891864e-01,
        7.81587279e-37,  4.48979726e-19,  4.48979726e-19, -4.77709577e-18,
        5.38916155e-18,  4.18505096e-35, -9.71014669e-35, -3.46500427e-18,
       -9.71014669e-35, -2.31455024e-01, -2.31455024e-01,  2.67261240e-01,
        1.55034890e-18, -1.59249446e-35,  5.93080351e-36, -1.48171862e-18,
        5.93080351e-36, -2.31455024e-01, -2.31455024e-01, -2.67261240e-01,
        4.00891864e-01,  1.09472937e-34,  5.59334602e-35, -1.33630620e-01,
        1.10528443e-34, -2.67261240e-01,  2.67261240e-01, -5.69100531e-19])

# Open pentagram

In [7]:
N = 5
pentagram_links = [[0,4],[1,8],[2,12],[3,16],[5,9],[6,13],[7,17],[10,14],[11,18],[15,19]]
link_states = [np.array([0,1/np.sqrt(2),-1/np.sqrt(2),0]).reshape((2,2)) for i in range(2*N)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))
    
l=[]
for link in link_states:
    l.append(tn.Node(link, name='L'+str(i)))
    
for i,(p,q) in enumerate(pentagram_links):
    n[p//4][p%4+1]^l[i][0]
    if (q//4)<4:
        l[i][1]^n[q//4][q%4+1]

    
state_open_pentagram = tn.contractors.auto(n+l, output_edge_order=[nn.edges[0] for nn in n]+[l[3].edges[1],l[6].edges[1],l[8].edges[1],l[9].edges[1]])
state_open_pentagram = norm(state_open_pentagram)

In [8]:
state_open_pentagram.tensor.reshape(2**8)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.00445933e-01, -2.00445933e-01,  0.00000000e+00,
        0.00000000e+00, -2.00445933e-01,  2.00445933e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.31455027e-01,
        0.00000000e+00, -1.15727512e-01, -1.15727512e-01,  0.00000000e+00,
        0.00000000e+00, -1.15727512e-01, -1.15727512e-01,  0.00000000e+00,
        2.31455027e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.98146682e-18, -2.98146682e-18,  0.00000000e+00,
        0.00000000e+00,  2.98146682e-18,  2.98146682e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.08693648e-18,
        0.00000000e+00,  

# Dekagram

In [9]:
state_open_pentagram_2 = tn.replicate_nodes([state_open_pentagram], conjugate=False)[0]

In [10]:
for i in range(2):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_pentagram[4]^n[0][1]
state_open_pentagram[5]^n[0][2]
state_open_pentagram_2[4]^n[0][3]
state_open_pentagram_2[5]^n[0][4]

state_open_pentagram[6]^n[1][1]
state_open_pentagram[7]^n[1][2]
state_open_pentagram_2[6]^n[1][3]
state_open_pentagram_2[7]^n[1][4]

state_dekagram = tn.contractors.auto([state_open_pentagram, n[0], n[1], state_open_pentagram_2], output_edge_order=
                                                                                        state_open_pentagram[:4]+
                                                                                        [n[0][0],n[1][0]]+
                                                                                        state_open_pentagram_2[:4])
state_dekagram = norm(state_dekagram)

In [11]:
state_dekagram.tensor.reshape(2**10)

array([ 3.04933923e-01,  0.00000000e+00, -5.28554556e-35, ...,
        7.18710013e-20,  1.43742005e-19, -2.65919206e-36])

In [23]:
# non zero elements of vector
for i,a in enumerate(state_dekagram.tensor.reshape(2**10)):
    if not np.isclose(a,0):
        print(format(i,'010b'),a)

0000000000 0.3049339225225648
0000000111 -0.17605367994544388
0000001011 -0.17605367994544385
0000001100 0.30493391985750734
0000001111 0.20328927812829994
0001110001 0.10164463906415004
0001110110 -0.05868455946892411
0001110111 0.06776309211719835
0001111010 -0.05868455946892411
0001111011 -0.06776309211719833
0001111101 -0.033881546058599166
0001111110 -0.06776309211719835
0110110001 -0.05868455946892411
0110110110 0.0338815460585992
0110110111 -0.03912303930402236
0110111010 0.0338815460585992
0110111011 0.03912303930402236
0110111101 0.019561519652011174
0110111110 0.039123039304022375
0111000000 -0.1760536799454439
0111000111 0.10164463817579757
0111001011 0.10164463817579757
0111001100 -0.1760536784067723
0111001111 -0.11736911791206713
0111110001 0.06776309211719835
0111110110 -0.03912303930402236
0111110111 0.04517539434997558
0111111010 -0.03912303930402236
0111111011 -0.04517539434997558
0111111101 -0.022587697174987785
0111111110 -0.04517539434997557
1010110001 -0.058684559

# Hexagram

In [26]:
state_open_node2 = tn.replicate_nodes([state_open_node], conjugate=False)[0]

In [27]:
links = [tn.Node(link) for i in range(4)]

n=[]
for i in range(4):
    n.append(tn.Node(iota, name='N'+str(i)))

state_open_node[1]^n[0][1]
state_open_node[2]^n[1][1]
state_open_node[3]^n[2][1]
state_open_node[4]^n[3][1]

state_open_node2[1]^n[1][2]
state_open_node2[2]^n[0][2]
state_open_node2[3]^n[3][2]
state_open_node2[4]^n[2][2]

links[0][0]^n[0][3]
links[0][1]^n[1][3]

links[1][0]^n[2][3]
links[1][1]^n[3][3]

links[2][0]^n[0][4]
links[2][1]^n[3][4]

links[3][0]^n[1][4]
links[3][1]^n[2][4]

state = tn.contractors.auto([state_open_node,state_open_node2]+n+links, output_edge_order=[state_open_node[0],n[0][0],n[1][0],state_open_node2[0],n[2][0],n[3][0]])
state_hexagram = norm(state)

In [28]:
state_hexagram.tensor.reshape(2**6)

array([-2.63342585e-01, -1.26063593e-18, -4.51610686e-18, -2.63342583e-01,
        2.85391704e-36,  1.07440648e-36, -1.94753376e-36, -1.52868484e-35,
       -6.64917179e-35, -1.48210821e-35, -3.70328323e-35, -9.98165862e-36,
       -2.20689373e-18,  1.52040910e-01,  1.52040910e-01,  1.75561720e-01,
        2.70104708e-35, -3.12611010e-35,  3.77494617e-35, -4.02651610e-35,
       -6.93103664e-19,  1.52040910e-01,  1.52040910e-01,  1.75561720e-01,
       -2.63342583e-01,  4.23350271e-18,  3.66183895e-18, -2.63342580e-01,
       -3.42251602e-18,  1.75561720e-01,  1.75561720e-01,  2.02721212e-01,
        2.70104708e-35, -3.12611010e-35,  3.77494617e-35, -4.02651610e-35,
       -2.63342583e-01, -3.93186543e-19,  1.57903230e-19,  8.77808601e-02,
        3.77307554e-19,  1.52040910e-01,  1.52040910e-01, -1.75561720e-01,
       -1.90872595e-18,  1.75561720e-01, -1.75561720e-01, -2.63079304e-18,
        3.77307554e-19,  1.52040910e-01,  1.52040910e-01, -1.75561720e-01,
        1.90872595e-18, -